In [0]:
import random
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report


In [0]:
def model_score(x_train, y_train, x_test, y_test, classes = 'binary'):
    
    #x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    model = DecisionTreeClassifier(random_state = 101)
    #print(type(model))
    model.fit(x_train,y_train)
    
    test_predictions = model.predict(x_test)
    if classes == 'multi':
        return f1_score(y_test, test_predictions, average = 'macro')
    else:
        return f1_score(y_test, test_predictions)

def create_random(n_features):
    n_chosen = random.randint(1,n_features-1)
    features = [0]*n_features
    for i in range(n_chosen):
        feat = random.randint(0,n_features-1)
        features[feat] = 1
    #features = np.random.randint(2,size=n_features)
    return features

class Optimizer():

    def __init__(self, X_train, y_train, X_test, y_test, retain=0.4, random_select=0.1, mutate_chance=0.6):
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.mutate_chance = mutate_chance
        self.random_select = random_select
        self.retain = retain
        self.graded = []

    def create_population(self, count):
        
        pop = []
        for _ in range(0, count):
            n_features = self.X_train.shape[1]
            feature_list = create_random(n_features)

            pop.append(feature_list)

        return pop
    
    def eval_pop(self, pop, classes ):
        self.graded = []
        
        alpha = 0.6
        for feature_array in pop:
            n = len(feature_array)
            feature_index = [i for i in range(len(feature_array)) if feature_array[i] > 0]
            #print(feature_index)
            f1_score = model_score(self.X_train[:,feature_index], self.y_train, self.X_test[:,feature_index], self.y_test, classes)
            fitness_score = alpha*f1_score + (1-alpha)*(n-len(feature_index))/n
            #print([fitness_score,f1_score, len(feature_index)])
            self.graded.append([fitness_score,f1_score, feature_array])
    
    def breed(self, mother, father):
        children = []
        for _ in range(2):

            child = [None]*len(mother)
            
            for param in range(len(mother)):
                child[param] = random.choice([mother[param], father[param]])

            if self.mutate_chance > random.random():
                child = self.mutate(child)

            children.append(child)

        return children

    def mutate(self, child):
        
        no_of_mutations = random.randint(0,len(child)-1)
        for i in range(no_of_mutations):
            mutation = random.randint(0,len(child)-1)
            child[mutation] = 0 if child[mutation]==1 else 1

        return child

    def evolve(self, pop):

        self.graded = [x[2] for x in sorted(self.graded, key=lambda x: x[0], reverse=True)]

        retain_length = int(len(self.graded)*self.retain)

        parents = self.graded[:retain_length]

        for individual in self.graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []

        while len(children) < desired_length:

            male = random.randint(0, parents_length-1)
            female = random.randint(0, parents_length-1)

            if male != female:
                male = parents[male]
                female = parents[female]

                babies = self.breed(male, female)

                for baby in babies:

                    if len(children) < desired_length and baby!= male and baby != female:
                        children.append(baby)

        parents.extend(children)

        return parents



def generate(generations, population, X_train, y_train, X_test, y_test, classes = 'binary'):
    
    optimizer = Optimizer(X_train, y_train, X_test, y_test)
    pop = optimizer.create_population(population)

    for i in range(generations):
        print("***Doing generation %d of %d***" %(i + 1, generations))
        optimizer.eval_pop(pop, classes)

        if i != generations - 1:
            pop = optimizer.evolve(pop)
            
    #print(optimizer.graded)
    final = [[x[1],x[2]] for x in sorted(optimizer.graded, key=lambda x: x[0], reverse=True)]
    final_pop = [[x[1],np.count_nonzero(x[2])] for x in sorted(optimizer.graded, key=lambda x: x[0], reverse=True)]
    print(final_pop)
    return final

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X_train=np.load("/content/drive/My Drive/Major2/X_train.npy")
y_train=np.load("/content/drive/My Drive/Major2/y_train.npy")
X_test=np.load("/content/drive/My Drive/Major2/X_test.npy")
y_test=np.load("/content/drive/My Drive/Major2/y_test.npy")

In [0]:
X_test.shape

(360, 208, 176)

In [0]:
X_train = X_train.reshape(840, 208*176)
X_test = X_test.reshape(360, 208*176)


In [0]:
final = generate(100,30, X_train, y_train, X_test, y_test, classes='multi')
res = final[0][1]
res_index = [i for i in range(len(res)) if res[i] > 0]

***Doing generation 1 of 100***
***Doing generation 2 of 100***
***Doing generation 3 of 100***
***Doing generation 4 of 100***
***Doing generation 5 of 100***
***Doing generation 6 of 100***
***Doing generation 7 of 100***
***Doing generation 8 of 100***
***Doing generation 9 of 100***
***Doing generation 10 of 100***
***Doing generation 11 of 100***
***Doing generation 12 of 100***
***Doing generation 13 of 100***
***Doing generation 14 of 100***
***Doing generation 15 of 100***
***Doing generation 16 of 100***
***Doing generation 17 of 100***
***Doing generation 18 of 100***
***Doing generation 19 of 100***
***Doing generation 20 of 100***
***Doing generation 21 of 100***
***Doing generation 22 of 100***
***Doing generation 23 of 100***
***Doing generation 24 of 100***
***Doing generation 25 of 100***
***Doing generation 26 of 100***
***Doing generation 27 of 100***
***Doing generation 28 of 100***
***Doing generation 29 of 100***
***Doing generation 30 of 100***
***Doing generation

In [0]:
len(res_index)

7090

In [0]:
X_train=X_train[:,res_index]
X_test=X_test[:,res_index]

In [0]:
X_test.shape

(360, 7090)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
print(neigh.score(X_test,y_test))
y_pred=neigh.predict(X_test)

0.9666666666666667


In [0]:
classifier = SVC()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
##accuracy_score(y_test, y_pred)
classifier.score(X_test,y_test)

0.825

In [0]:
y_train.shape

(840,)

In [0]:
np.save("X_train_GA.npy",X_train)
np.save("X_test_GA.npy",X_test)
np.save("y_train_GA.npy",y_train)
np.save("y_test_GA.npy",y_test)

In [0]:
from google.colab import files
files.download('X_train_GA.npy')
files.download('X_test_GA.npy')
files.download('y_train_GA.npy')
files.download('y_test_GA.npy')